In [ ]:
#1) COLLECT THE COMPLEXES DATA FROM THE API ENDPOINTS-
import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/complexes.json?api_key=QMBNelaHxCHPZs74ym4Ix6FML9dpVRSpHb4ACxLE"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)
data = response.json()

In [ ]:
data["complexes"][-1]

In [ ]:
import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/complexes.json?api_key=QMBNelaHxCHPZs74ym4Ix6FML9dpVRSpHb4ACxLE"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    ab = []
    
    # Loop through complexes
    for cmplx in data.get('complexes', []):
        for venue in cmplx.get('venues', []):  
            ab.append({
                'complex_id': cmplx.get('id'),
                'complex_name': cmplx.get('name'),
                'venue_id': venue.get('id'),
                'venue_name': venue.get('name'),
                'city_name': venue.get('city_name'),
                'country_name': venue.get('country_name'),
                'country_code': venue.get('country_code'),
                'timezone': venue.get('timezone')
            })
    
    # Print the result
    for item in ab:
        print(item)

else:
    print(f"Failed to fetch data: {response.status_code} - {response.text}")


In [ ]:
import pandas as pd
pf=pd.DataFrame(ab)
pf

In [1]:
import mysql.connector

conn = mysql.connector.connect(
    host="localhost",  
    user="root", 
    password="Neethiuma@123",  
    database="sportsanalytics"  
)

In [10]:
# Establishing the connection
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS complex (
    complex_id VARCHAR(50),
    complex_name VARCHAR(100)
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS venue (
    venue_id VARCHAR(50),
    venue_name VARCHAR(100),
    city_name VARCHAR(50),
    country_name VARCHAR(100),
    country_code VARCHAR(5),
    timezone VARCHAR(100),
    complex_id VARCHAR(50)
);
''')

# Insert data into complex table
for index, row in pf.iterrows():
    cursor.execute('''
    INSERT INTO complex (complex_id, complex_name)
    VALUES (%s, %s);
    ''', (row['complex_id'], row['complex_name']))

# Insert data into venue table
for index, row in pf.iterrows():
    cursor.execute('''
    INSERT INTO venue (venue_id, venue_name, city_name, country_name, country_code, timezone, complex_id)
    VALUES (%s, %s, %s, %s, %s, %s, %s);
    ''', (row['venue_id'], row['venue_name'], row['city_name'], row['country_name'], row['country_code'], row['timezone'], row['complex_id']))

# Commit changes and close the connection
conn.commit()
cursor.close()
conn.close()

print("Data has been inserted into MySQL!")


Data has been inserted into MySQL!


In [2]:
from tabulate import tabulate

In [3]:
import mysql.connector
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Neethiuma@123",
    database="sportsanalytics"
)
cursor = conn.cursor()

In [4]:
#1.List all venues along with their associated complex name
cursor.execute(
    '''SELECT v.venue_name, c.complex_name 
FROM venue v 
JOIN complex c ON v.complex_id = c.complex_id limit 100;
    '''             
)
results = cursor.fetchall()
headers = ["venue_name", "complex_name"]
table = tabulate(results, headers, tablefmt="grid")
print(table)

+----------------+----------------------------+
| venue_name     | complex_name               |
+================+============================+
| Cancha Central | Nacional                   |
+----------------+----------------------------+
| Court One      | Estadio de la Cartuja      |
+----------------+----------------------------+
| Centre Court   | Estadio de la Cartuja      |
+----------------+----------------------------+
| Court One      | Estadio de la Cartuja      |
+----------------+----------------------------+
| Centre Court   | Estadio de la Cartuja      |
+----------------+----------------------------+
| TC Dynamo      | Sibur Arena                |
+----------------+----------------------------+
| Sibur Arena    | Sibur Arena                |
+----------------+----------------------------+
| CENTER COURT   | Sibur Arena                |
+----------------+----------------------------+
| COURT 1        | Sibur Arena                |
+----------------+----------------------

In [5]:
#2.Count the number of venues in each complex
cursor.execute(
    '''SELECT count(v.venue_name) as count_venue, c.complex_name 
FROM venue v 
JOIN complex c ON v.complex_id = c.complex_id
group by c.complex_name;
    '''             
)
results = cursor.fetchall()
headers = ["count_venue", "complex_name"]
table = tabulate(results, headers, tablefmt="grid")
print(table)

+---------------+------------------------------------------------------------+
|   count_venue | complex_name                                               |
+===============+============================================================+
|             1 | Nacional                                                   |
+---------------+------------------------------------------------------------+
|             4 | Estadio de la Cartuja                                      |
+---------------+------------------------------------------------------------+
|            16 | Sibur Arena                                                |
+---------------+------------------------------------------------------------+
|            25 | Complexo de Tenis do Jamor                                 |
+---------------+------------------------------------------------------------+
|            36 | Ostravar Arena                                             |
+---------------+-----------------------------------

In [6]:
#3.Get details of venues in a specific country (e.g., Chile)
cursor.execute(
    '''SELECT * 
FROM venue 
WHERE country_name = 'Chile';
    '''             
)
results = cursor.fetchall()
headers = [""]
table = tabulate(results, headers, tablefmt="grid")
print(table)

+----------------+-------------------------+--------------+-------+-----+------------------+------------------+
|                |                         |              |       |     |                  |                  |
+================+=========================+==============+=======+=====+==================+==================+
| sr:venue:70045 | Cancha Central          | Santiago     | Chile | CHL | America/Santiago | sr:complex:705   |
+----------------+-------------------------+--------------+-------+-----+------------------+------------------+
| sr:venue:67857 | Cancha Central          | Temuco       | Chile | CHL | America/Santiago | sr:complex:11769 |
+----------------+-------------------------+--------------+-------+-----+------------------+------------------+
| sr:venue:67859 | Cancha 1                | Temuco       | Chile | CHL | America/Santiago | sr:complex:11769 |
+----------------+-------------------------+--------------+-------+-----+------------------+------------

In [7]:
#4.Identify all venues and their timezones
cursor.execute(
    '''SELECT venue_name, timezone 
FROM venue;
    '''             
)
results = cursor.fetchall()
headers = ["venue_name", "timezone"]
table = tabulate(results, headers, tablefmt="grid")
print(table)

+-----------------------------------------+--------------------------------+
| venue_name                              | timezone                       |
+=========================================+================================+
| Cancha Central                          | America/Santiago               |
+-----------------------------------------+--------------------------------+
| Centre Court                            | Europe/Madrid                  |
+-----------------------------------------+--------------------------------+
| Court One                               | Europe/Madrid                  |
+-----------------------------------------+--------------------------------+
| COURT 1                                 | Europe/Moscow                  |
+-----------------------------------------+--------------------------------+
| CENTER COURT                            | Europe/Moscow                  |
+-----------------------------------------+--------------------------------+

In [8]:
#5.Find complexes that have more than one venue
cursor.execute(
    '''select count(venue_id) as cnt,complex_name from complex a join venue b on a.complex_id = b.complex_id
group by complex_name
having cnt >1

    '''             
)
results = cursor.fetchall()
headers = ["cnt", "complex_name"]
table = tabulate(results, headers, tablefmt="grid")
print(table)

+-------+------------------------------------------------------------+
|   cnt | complex_name                                               |
+=======+============================================================+
|     4 | Estadio de la Cartuja                                      |
+-------+------------------------------------------------------------+
|    16 | Sibur Arena                                                |
+-------+------------------------------------------------------------+
|    25 | Complexo de Tenis do Jamor                                 |
+-------+------------------------------------------------------------+
|    36 | Ostravar Arena                                             |
+-------+------------------------------------------------------------+
|    64 | Varazdin Arena                                             |
+-------+------------------------------------------------------------+
|    25 | Arena Montpellier                                          |
+-----

In [9]:
#6.List venues grouped by country
cursor.execute(
    '''select venue_name,country_name from venue group by 1,2

    '''             
)
results = cursor.fetchall()
headers = ["venue_name", "country_name"]
table = tabulate(results, headers, tablefmt="grid")
print(table)

+-----------------------------------------+----------------------+
| venue_name                              | country_name         |
+=========================================+======================+
| Cancha Central                          | Chile                |
+-----------------------------------------+----------------------+
| Centre Court                            | Spain                |
+-----------------------------------------+----------------------+
| Court One                               | Spain                |
+-----------------------------------------+----------------------+
| COURT 1                                 | Russia               |
+-----------------------------------------+----------------------+
| CENTER COURT                            | Russia               |
+-----------------------------------------+----------------------+
| Sibur Arena                             | Russia               |
+-----------------------------------------+-------------------

In [10]:
#7.Find all venues for a specific complex (e.g., Nacional)
cursor.execute(
    '''SELECT v.venue_name,v.venue_id
FROM venue v 
JOIN complex c ON v.complex_id = c.complex_id 
WHERE c.complex_name = 'Nacional';
    '''             
)
results = cursor.fetchall()
headers = ["venue_name", "venue_id"]
table = tabulate(results, headers, tablefmt="grid")
print(table)

+----------------+----------------+
| venue_name     | venue_id       |
+================+================+
| Cancha Central | sr:venue:70045 |
+----------------+----------------+
